# 📈 Pipeline de Dados de Criptomoedas - Binance API
**Autor:** Aldo Bernardi  
**Última atualização:** {data_atual}

---
## Objetivo
Extrair dados históricos de criptomoedas diretamente da Binance API para análise técnica e construção de dashboards.

In [ ]:
# Configuração inicial
!pip install ccxt pandas plotly kaleido --quiet
import ccxt
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import os

# Parâmetros ajustáveis
SYMBOLS = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT']
TIMEFRAME = '1d'
DAYS_BACK = 180
CACHE_DIR = './data'

# Criar diretório de cache se não existir
os.makedirs(CACHE_DIR, exist_ok=True)

## 🔄 Extração de Dados com Tratamento de Erros

In [ ]:
def get_binance_data(symbol, days=DAYS_BACK, retries=3):
    """Extrai dados OHLCV com sistema de retry e cache"""
    cache_file = f"{CACHE_DIR}/{symbol.replace('/', '')}_{TIMEFRAME}.csv"
    
    # Verificar cache
    if os.path.exists(cache_file):
        df = pd.read_csv(cache_file, parse_dates=['timestamp'], index_col='timestamp')
        return df
    
    binance = ccxt.binance({'enableRateLimit': True})
    
    for attempt in range(retries):
        try:
            since = binance.parse8601((datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d 00:00:00'))
            ohlcv = binance.fetch_ohlcv(symbol, TIMEFRAME, since=since)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            
            # Salvar em cache
            df.to_csv(cache_file)
            return df
        except Exception as e:
            print(f"Tentativa {attempt + 1} falhou para {symbol}: {str(e)}")
            if attempt == retries - 1:
                raise
            time.sleep(2)

## 📊 Visualização Interativa

In [ ]:
def create_candlestick_chart(df, symbol):
    """Gera gráfico de candlestick com Plotly"""
    fig = go.Figure(data=[go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name=symbol
    )])
    
    fig.update_layout(
        title=f"Análise de {symbol} - Últimos {DAYS_BACK} dias",
        xaxis_title='Data',
        yaxis_title='Preço (USDT)',
        template='plotly_dark',
        hovermode="x unified"
    )
    
    return fig

# Exemplo de uso
btc_data = get_binance_data('BTC/USDT')
fig = create_candlestick_chart(btc_data, 'BTC/USDT')
fig.show()

## 🚀 Exportação para Dashboard HTML

In [ ]:
def export_dashboard():
    """Gera dashboard HTML com múltiplos ativos"""
    figs = []
    for symbol in SYMBOLS:
        df = get_binance_data(symbol)
        figs.append(create_candlestick_chart(df, symbol))
    
    # Layout do dashboard
    dashboard = go.Figure()
    for fig in figs:
        for trace in fig.data:
            dashboard.add_trace(trace)
    
    dashboard.update_layout(
        title='Dashboard Criptomoedas - Binance Data',
        grid={'rows': len(SYMBOLS), 'columns': 1, 'pattern': "independent"},
        height=300 * len(SYMBOLS)
    )
    
    # Exportar
    dashboard.write_html("./assets/crypto_dashboard.html",
                       full_html=True,
                       include_plotlyjs='cdn')

# Executar exportação
export_dashboard()

## 📌 Próximos Passos
- Adicionar indicadores técnicos (RSI, MACD)
- Implementar alertas de preço
- Conectar com banco de dados para histórico completo